Importating libraires 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
from scipy.stats import norm

Analytical pricing (using Black-Scholes formula)

In [2]:
def getd1andd2(spot, strike, maturity, sigma, r, div):
    if maturity!=0 and sigma!=0:
        d1=(np.log(spot/strike)+maturity*(r-div+0.5*sigma**2))/(sigma*np.sqrt(maturity))
        d2=d1-sigma*np.sqrt(maturity)
        return(d1,d2)
    else:
         return(np.nan)
def getprices(spot,div,maturity,d1,d2,r,strike):
    C=spot*np.exp(-div*maturity)*norm.cdf(d1)-strike*np.exp(-r*maturity)*norm.cdf(d2)
    P=C-spot*np.exp(-div*maturity)+strike*np.exp(-r*maturity)
    return(C,P)
#exemple SP500
S0=690
K=700
T=0.25
sigma=0.12
r=0.04
div=0.0115
d1,d2=getd1andd2(S0,K,T,sigma,r,div)
call,put=getprices(S0,div,T,d1,d2,r,K)

Montecarlo Pricing

In [6]:
def getprices2(spot, strike, r, div, sigma, maturity):
    tirages=np.random.normal(0,1,100000)
    payoffcall=[]
    payoffput=[]
    for tirage in tirages:
        S1=spot*np.exp((r-div-0.5*sigma**2)*maturity+sigma*np.sqrt(maturity)*tirage)
        payoffcall.append(max(S1-strike,0))
        payoffput.append(max(strike-S1,0))
    C=np.exp(-r*maturity)*sum(payoffcall)/len(payoffcall)
    P=np.exp(-r*maturity)*sum(payoffput)/len(payoffput)
    return(C,P)
callM,putM=getprices(S0,div,T,d1,d2,r,K)

Computing greeks

In [19]:
def greeks_call(spot, strike, maturity, sigma, r, div):
    d1, d2=getd1andd2(spot, strike, maturity, sigma, r, div)
    delta=np.exp(-div*maturity)*norm.cdf(d1)
    def getgamma(spot, epsilon,strike, maturity, sigma, r, div):
        d1U,d2U=getd1andd2(spot+epsilon, strike, maturity, sigma, r, div)
        d1L,d2L=getd1andd2(spot-epsilon, strike, maturity, sigma, r, div)
        callU,_=getprices(spot+epsilon,div,maturity,d1U,d2U,r,strike)
        callL,_=getprices(spot-epsilon,div,maturity,d1L,d2L,r,strike)
        call,_=getprices(spot,div,maturity,d1,d2,r,strike)
        gamma=(callU-2*call+callL)/epsilon**2
        return(gamma)
    gamma=getgamma(spot, 0.0001,strike, maturity, sigma, r, div)
    
    def getvega(spot, epsilon,strike, maturity, sigma, r, div,d1,d2):
        d1U,d2U=getd1andd2(spot, strike, maturity, sigma+epsilon, r, div)
        d1L,d2L=getd1andd2(spot, strike, maturity, sigma-epsilon, r, div)
        callU,_=getprices(spot,div,maturity,d1U,d2U,r,strike)
        callL,_=getprices(spot,div,maturity,d1L,d2L,r,strike)
        call,_=getprices(spot,div,maturity,d1,d2,r,strike)
        vega=(callU-callL)/(2*epsilon)
        return(vega)
    vega=getvega(spot, 0.0001,strike, maturity, sigma, r, div,d1,d2)
    
    def gettheta(spot, epsilon,strike, maturity, sigma, r, div,d1,d2):
        d1U,d2U=getd1andd2(spot, strike, maturity+epsilon, sigma, r, div)
        d1L,d2L=getd1andd2(spot, strike, maturity-epsilon, sigma, r, div)
        callU,_=getprices(spot,div,maturity+epsilon,d1U,d2U,r,strike)
        callL,_=getprices(spot,div,maturity-epsilon,d1L,d2L,r,strike)
        call,_=getprices(spot,div,maturity,d1,d2,r,strike)
        theta=(callU-callL)/(2*epsilon)
        return(theta)
    theta=gettheta(spot, 0.0001,strike, maturity, sigma, r, div,d1,d2)

    def getrho(spot, epsilon,strike, maturity, sigma, r, div,d1,d2):
        d1U,d2U=getd1andd2(spot, strike, maturity, sigma, r+epsilon, div)
        d1L,d2L=getd1andd2(spot, strike, maturity, sigma, r-epsilon, div)
        callU,_=getprices(spot,div,maturity,d1U,d2U,r+epsilon,strike)
        callL,_=getprices(spot,div,maturity,d1L,d2L,r-epsilon,strike)
        call,_=getprices(spot,div,maturity,d1,d2,r,strike)
        rho=(callU-callL)/(2*epsilon)
        return(rho)
    rho=getrho(spot, 0.0001,strike, maturity, sigma, r, div,d1,d2)

    return(delta, gamma, vega, theta, rho)
#exemple SP500
S0=690
K=700
T=0.25
sigma=0.12
r=0.04
div=0.0115
delta, gamma, vega, theta, rho=greeks_call(S0, K, T, sigma, r, div)
print(f"Delta:{delta}")
print(f"Gamma:{gamma}")
print(f"Vega:{vega}")
print(f"Theta:{theta}")
print(f"Rho:{rho}")

Delta:0.4623902630074133
Gamma:0.009566747394273989
Vega:136.6721111949687
Theta:41.328636951334374
Rho:76.22747681978126
